# Our GPS System

In this notebook I am going to explain you the basics of our GPS system, how it is build and how to listen to what it broadcasts.


## What is GPS?

The formal definition:
> The Global Positioning System (GPS) is an utility that provides users with positioning, navigation, and timing services. The system consists of three segments: the space segment, the control segment, and the user segment.

The informal deffinition:
> Someting that determines where you are.

## Architecture

Our GPS is a camera-based centralized system wich is able to detect the position of your car by detecting the position of the attached Aruco marker. 

It has 3 major components:
- **client car** = a competition car with a Aruco marker attached to it

- **node** = a RaspberryPi with a PiCamera attached to it, placed at a certain height that allows the surveilance of a certain region of the competition track

- **server node** = a node to whom the client car and all the other nodes are connected. It collects and processes all the data from the other nodes. It brodcasts 2 messages at a certain rate: that it is the server and the ID, position and orientation of your Aruco Marker.  

All of th eabove components have to be connected to the same network. To access the information provided by the system, you will have to authenticate from `your car` to the `server node`. The server node can change at every moment, being able to be replaced by any other node wich will become the new server. The server combines all the coordinate systems of the nodes in one unified coordinate system and computes the exact position of your car. 

The messages from the server are transmitted as follow:
    - the 'I am the server' message is broadcasted on the `NEGOTIATION_PORT` by the server to all the devices connected to the network via the UDP protocol
    - the message with the ID, position and orientation will be transmitted to your car 

###  How are the position and orientation computed?

 - **position** is denoted by the bottom left of your Aruco marker. The position will be transmitted in the global coordinate system

 - **orientation** is a vector specified by the bottom-corners of the Aruco marker. The orientation vector is perpendicular to the right side of the car.

<img src="./images/gps2.png">

### Technical specification:
- **accuracy:** 5-10cm
- **position transmission rate:** 1 Hz
- **server IP tasnsmission rate:** 0.52 Hz (once every 1.9 seconds)



**`Note:`** The freq. of the transmission could be changed in the feauture.

## BFMC_GPS package

 This piece of software will come in a package `BFMC_GPS`. 


### Client side example code

In this section, I will show a demo code for the client side, by this, I mean how to connect to the server and listen for its messages. This file can be found in `BFMC/client/CarClient.py`.

This part should run on your car.

The application is splitted into 3 threads: 
    - one resonsable for listening to the "I am the server" message and tring to authenticate to the server node
    - one resposible for listening to the data that the server transmits (position and orientation)
    - main thread, where the other threads launch

Import the libraries that we will use:

In [1]:
import os
import sys

from math       import  *
from random     import  *
from socket     import  *
from threading  import  Thread
from time       import  *

We can use the `socket` library in order for us to see the local IP and the hostname:

(only compatible on Unix systems)

In [2]:
gw =  os.popen("ip -4 route show default").read().split()

s  =  socket(AF_INET,SOCK_DGRAM)
s.connect((gw[2],0))

HOST_NAME = gethostname()
HOST_IP   = s.getsockname()[0]

s.close()

print("Host name:"+str(HOST_NAME)+" IP:"+str(HOST_IP))

IndexError: list index out of range

After that, we define and configure some global parameters:

In this area we will specify :
   - the data regarding your car(ID, IP, and HOSTNAME)
   - we will set the ports for negociation, for subscription and for communication. **`Note`** this should be left unchanged.
   - the car initial position and orientation
   - some utility flags

In [ ]:
THIS_CAR_ID                 =  7                        # car marker ID attached to the car
MAX_WAIT_TIME_FOR_SERVER    =  5

#communication parameters
SERVER_IP                   =  None                       # current server ip
NEGOTIATION_PORT            =  12346                      # base port that we will use(listen for server)
CAR_SUBSCRITPION_PORT       =  NEGOTIATION_PORT + 2       # auth port      CAR -> GPS position
CAR_COMMUNICATION_PORT      =  CAR_SUBSCRITPION_PORT + 2  # port           GPS -> CAR
G_Socket_Poz                =  socket()                   # socket used for communication

NEW_SERVER_IP               =  False
car_id                      =  THIS_CAR_ID

#flags                                                    
ID_SENT_FLAG                =  False                      # used for checking if the ip was sent for auth
START_UP                    =  True                        
RUN_CARCLIENT               =  True                       

#car related data
carOrientation              =  0+0j                       # car orientation  (complex number)
carPos                      =  0+0j                       # car position (complex number)

This function will run in a thread. It will be responsible for listening for the server's IP.

In [ ]:
def GetServer():
    '''
        Method used for listening for the server IP and trying to register to it. Note that the server may change.
    '''

    global SERVER_IP
    global NEGOTIATION_PORT
    global NEW_SERVER_IP
    global G_Socket_Poz
    global START_UP
    global ID_SENT_FLAG
    global RUN_CARCLIENT
    
    while RUN_CARCLIENT:
        try:
            s = socket(AF_INET, SOCK_DGRAM)
            s.bind(('', NEGOTIATION_PORT))
            s.setsockopt(SOL_SOCKET, SO_BROADCAST, 1)

            t       =  time()
            server  =  []

            # Listen for server broadcast. 
            s.settimeout(MAX_WAIT_TIME_FOR_SERVER)

            # Receive data from the socket. Buffer size = 1500 bytes
            data, server_ip = s.recvfrom(1500, 0)

            # Get server IP                          
            if server_ip[0] != SERVER_IP:
                # new server
                SendIDToServer(server_ip[0])
                NEW_SERVER_IP   =   True
                SERVER_IP       =   server_ip[0] # server is alive
                START_UP        =   False

            else:
                # old server
                if ID_SENT_FLAG == False:
                    SendIDToServer(server_ip[0])
                    print("Subscribe @ GPS server")
                NEW_SERVER_IP = False
                # Server beacon received
                s.close()
            
        except Exception as e:
            SERVER_IP = None # Server is dead.
            if START_UP == False and SERVER_IP == None and NEW_SERVER_IP == True:
                G_Socket_Poz.close()
                G_Socket_Poz = None
                print("Socket from get position server closed!")
                
            print ("Not connected to server! IP: " + str(SERVER_IP) + 
                    "! Error:" + str(e)
                    )

            s.close()

This is an utility function that will send your IP and Aruco Markers ID to the server.

In [ ]:
def SendIDToServer(new_server_ip):
    '''Send the car id to server for identification. 
    
    Arguments:
        new_server_ip {[type]} -- the ip of the new server
    '''

    global SERVER_IP
    global CAR_SUBSCRITPION_PORT
    global THIS_CAR_ID
    global ID_SENT_FLAG

    try:
        # Open connection to server.
        s = socket()         
        print("Vehicle " + str(THIS_CAR_ID) + 
                " subscribing to GPS server: " + str(new_server_ip) +
                ":"+ str(CAR_SUBSCRITPION_PORT)
            )

        s.connect((new_server_ip, CAR_SUBSCRITPION_PORT))
        sleep(2) 
        car_id_str = str(THIS_CAR_ID)
        s.send(bytes(car_id_str, "UTF-8"))
        # Shut down conn. Further sends are disallowed.
        s.shutdown(SHUT_WR)
        s.close()
        print("Vehicle ID sent to server----------------------------")
        ID_SENT_FLAG = True

    except Exception as e:
        print("Failed to send ID to server, with error: " + str(e))
        s.close()
        ID_SENT_FLAG = False

This is the thread that will get you the information for your car from the server node:

The server will send you the message in the following form:
> id:{`<car_id>`};Pos:({`<real_pos>`}+{`<imag_pos>`}j);Azm:({`<real_azm>`}+{`<imag_azm>`}j)

Where :
    - <car_id> - int
    - <real_pos> - float with 2 decimals precission (position on X axis)
    - <imag_pos> - float with 2 decimals precission (position on Y axis)
    - <real_azm> - float with 2 decimals precission (first point of a vector denoting orientation)
    - <imag_azm> - float with 2 decimals precission (second first point of a vector denoting orientation)

In [ ]:
#================================ GET DATA ====================================
def GetPositionDataFromServer_Thread():
    '''
    Utility function for receiving the car position and orientation from the server.
    '''

    global car_id
    global carPos
    global carOrientation
    global SERVER_IP
    global NEW_SERVER_IP
    global THIS_CAR_ID
    global CAR_COMMUNICATION_PORT
    global G_Socket_Poz
    global RUN_CARCLIENT

    while RUN_CARCLIENT:
        if SERVER_IP != None: # If server alive/valid
            if NEW_SERVER_IP == True:
                try:
                    G_Socket_Poz.close()
                    G_Socket_Poz=None
                except:
                    # Do nothing.
                    print("Previous socket cound not be closed.")
            if G_Socket_Poz==None:
                # Server changed.
                try:
                    # If there is a GPS server available then open a socket and then wait for GPS data.
                    print("Attempting to create new socket to receive the position from server "+str(HOST_IP)+":"+str(CAR_COMMUNICATION_PORT))
                    #G_Socket_Poz = socket()   
                    G_Socket_Poz = socket(AF_INET, SOCK_STREAM)
                    # Set socket flag socket.SO_REUSEADDR for preventing "[Errno 98] Address already in use" error
                    # when restarting application 
                    G_Socket_Poz.setsockopt(SOL_SOCKET, SO_REUSEADDR, 1)
                    G_Socket_Poz.bind((HOST_IP, CAR_COMMUNICATION_PORT))      
                    G_Socket_Poz.listen(2)

                    NEW_SERVER_IP = False
                
                except Exception as e:
                    print("Creating new socket for get position from server " + str(SERVER_IP) + " failed with error: "+str(e))
                    sleep(1)
                    G_Socket_Poz = None
            
            if not G_Socket_Poz == None:
                # Server did not change.
                try:
                    c, addr =    G_Socket_Poz.accept()
                    # raw message   
                    data    =    str(c.recv(4096))   
                    # mesage parsing
                    car_id  =    int(data.split(';')[0].split(':')[1])
                    
                    if car_id == THIS_CAR_ID:
                        # Compute car position.
                        carPos = complex(float(data.split(';')[1].split(':')[1].split('(')[1].split('+')[0]), float(data.split(';')[1].split(':')[1].split('j')[0].split('+')[1]))
                        # Compute car orientation.
                        carOrientation = complex(float(data.split(';')[2].split(':')[1].split('(')[1].split('+')[0]), float(data.split(';')[2].split(':')[1].split('j')[0].split('+')[1]))
                        
                        print("id:" + str(car_id) + " -position: " + str(carPos) + " -orientation: " + str(carOrientation))
                    c.close()

                except Exception as e:
                        print("Receiving position data from server " + str(SERVER_IP) + " failed with error: " + str(e))
                        c.close()

This is the main thread, where all the other threads are launched.

In [ ]:
global RUN_CARCLIENT
print("First start up of vehicle client ...")

# thread responsible for subscribing to GPS server
connection_thread = Thread(target = GetServer)

# thread responsible for receiving GPS data from server
position_thread = Thread(target = GetPositionDataFromServer_Thread)

try:
    # start all threads
    connection_thread.start()
    position_thread.start()
    while RUN_CARCLIENT:
        print("Running")
        sleep(30)
except KeyboardInterrupt as e:
    RUN_CARCLIENT=False
    try:
        global G_Socket_Poz
        G_Socket_Poz.close()
        print("\nClient socket closed!")

    except:
        print("Cannot close client socket!")
        pass

    print("KeyboardInterrupt!!")
    pass

# The server-side of or GPS system

In order for you to be able to test the communication with our system, we have developed a simulated GPS that you should be able to run from your computer. 

#### `NOTE` IT IS A SIMULATION! IT WON"T SHOW YOU THE ACTUAL POSITION AND ORIENTATION OF YOUR CAR!


## How to run the simulated GPS

In this section I will show you how to launch the simulated GPS.
For this you wil have to launch 2 parts: the server side which will run on your computer and the client side which will run on the RaspberryPi.

1. For server-side:
    - copy the BFMC_GPS in your working directory on your computer (must be Unix):
    - go to BFMC_GPS directory
    - execute `python3 gps.py`

2. For the client-side:
    - copy the BFMC_GPS in your working directory on your car's RaspberryPi
    - go to BFMC_GPSclient directory
    - execute `python3 CarClient.py`
    
As from now, on the client side you should be able to see the received messages.

### Common problems
    - make sure that you run both sides on an Unix env(i.e. Ubuntu)
    - make sure that both component are connected to the same network

## Simulated part

In this part I am going to explain you who is responsible for sending simulated data and how to modify it as you please.

By default, the system is programmed to send you the coordinates for a circular trajectory. The car ID should be within range [0,30].

The code responsible for this is located in `BFMC_GPS/gps/SimulatorClient.py`

If you would like to send a custom trajectory via the GPS, you can do that by modifyng the following method:
    - SimulatorClient.points()
    
This should return a list of complex numbers generated according to your trajectory.